#### Import the libraries

In [1]:
# For cleaning and preparing the dataset
# -> dataframe manipulation
# -> text manipulation
# -> Web Scrapping

import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import re


# Module to serialize the content produced from the execution of the code

import pickle


# Module to monitor the progress of a python for loop

from tqdm import tqdm


# Module to manipulate text in python - NLTK package

import nltk
from nltk import word_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords


# Module to compute word vectorizers and compute the cosine distance

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_distances

# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -

#### Import the dataset

In [127]:
dataset = pd.read_pickle('D:\\GitHub-Thesis\\58,000 movies\\final_dataset_49393_movies_25012020.pkl') 

#previous versions "final_dataset_07112019", "final_dataset_22102019.pkl", dataset_part_1_17112019.pkl

dataset = dataset.reset_index(drop=True)

In [ ]:
#dataset = dataset.join(pd.DataFrame(dataset.Genres.values.tolist(), dataset.index).add_prefix('Genres_').fillna(0)) old versions

In [ ]:
#dataset = dataset.drop(['Genres_3', 'Genres_4', 'Genres_5', 'Genres_6', 'Genres_7', 'Genres_8'], axis=1) old versions

In [ ]:
dataset.head()

# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -

#### Cleaning the columns

##### Movie Title (25.10.2019)

##### Step 1: Clean the year (i.e 2001) from each title - 25.10.2019

In [128]:
exp = r'\(\d\d\d\d.'

dataset['title'] = dataset['title'].apply(lambda x: re.sub(exp,"",x).strip())

In [ ]:
# dataset['title'] = dataset['title'].apply(lambda x: x.replace(',', ''))

In [92]:
dataset.head()

,title,genres,rating,imdb_url,reviews_url,actors,plot,imdb_rating,director,reviews
0,Toy Story,"[Adventure, Animation, Children]",3.89,http://www.imdb.com/title/tt0114709/,http://www.imdb.com/title/tt0114709/reviews?sp...,"[Tom Hanks, Tim Allen, Don Rickles, Jim Varney...",\n A cowboy doll is profoun...,8.3,John Lasseter,[Andy's toys live a reasonable life of fun and...
1,Jumanji,"[Adventure, Children, Fantasy]",3.25,http://www.imdb.com/title/tt0113497/,http://www.imdb.com/title/tt0113497/reviews?sp...,"[Robin Williams, Jonathan Hyde, Kirsten Dunst,...",\n When two kids find and p...,7.0,Joe Johnston,[The smash hit 1995 film Jumanji- based on the...
2,Grumpier Old Men,"[Comedy, Romance]",3.17,http://www.imdb.com/title/tt0113228/,http://www.imdb.com/title/tt0113228/reviews?sp...,"[Walter Matthau, Jack Lemmon, Sophia Loren, An...",\n John and Max resolve to ...,6.7,Howard Deutch,[Sequels are rarely half as good as the origin...
3,Waiting to Exhale,"[Comedy, Drama, Romance]",2.87,http://www.imdb.com/title/tt0114885/,http://www.imdb.com/title/tt0114885/reviews?sp...,"[Whitney Houston, Angela Bassett, Loretta Devi...",\n Based on Terry McMillan'...,5.9,Forest Whitaker,[I read some of the other comments comcerning ...
4,Father of the Bride Part II,[Comedy],3.08,http://www.imdb.com/title/tt0113041/,http://www.imdb.com/title/tt0113041/reviews?sp...,"[Steve Martin, Diane Keaton, Martin Short, Kim...",\n George Banks must deal n...,6.0,Charles Shyer,"[I enjoyed this film, as I did Father of the B..."


##### Step 2: Replace (,The) & (,A) from the end of the string, at the front - 26.01.2020

In [129]:
dataset[dataset.director == "Guido Zurli"]

,title,genres,rating,imdb_url,reviews_url,actors,plot,imdb_rating,director,reviews
29885,Bersaglio altezza uomo,[Action],2.0,http://www.imdb.com/title/tt0078848/,http://www.imdb.com/title/tt0078848/reviews?sp...,"[Luc Merenda, Gabriella Giorgelli, Kadir Inani...",\nAdd a Plot »\n,5.5,Guido Zurli,[This is surely one of the lamest poliziottesc...
29886,The Mad Butcher,[Horror],2.0,http://www.imdb.com/title/tt0067799/,http://www.imdb.com/title/tt0067799/reviews?sp...,"[Victor Buono, Franca Polesello, Brad Harris, ...",\n After being released fro...,5.3,Guido Zurli,[When formerly respected local butcher Otto Le...
29887,Psychopath,"[Comedy, Crime]",2.5,http://www.imdb.com/title/tt0061983/,http://www.imdb.com/title/tt0061983/reviews?sp...,"[George Martin, Ingrid Schoeller, Karin Field,...",\n The hero steals the haul...,5.0,Guido Zurli,"[Little role for Klaus Kinski but funny, he's ..."
29888,Thompson 1880,[Western],0.0,http://www.imdb.com/title/tt0061090/,http://www.imdb.com/title/tt0061090/reviews?sp...,"[George Martin, Gia Sandri, José Bódalo, Osiri...",\nAdd a Plot »\n,6.3,Guido Zurli,[A western made during the boom of such movies...


#### Comment: As we can see two of the above titles have the inconcistencies mentioned. Thus, they should be fixed.

In [48]:
dataset_test_sql = dataset.iloc[0:1810]

dataset_test_sql.shape

(1810, 10)

In [53]:
def move_word(s, word, pos):
    split = s.split(',')
    if "The" in split:
        print(s)
        #split = s.split()
        split.insert(pos, split.pop(split.index(word)))
        print(split)
        return ' '.join(split)
    else:
        print(s)
        return ' '.join(split)

In [ ]:
def reorder(s, word, delta):
    words = s.split()
    oldpos = words.index(word)
    words.insert(oldpos+delta, words.pop(oldpos))
    return ' '.join(words)

In [54]:
# Dummy test dataset

# dataset_test_sql.title = dataset_test_sql.title.apply(lambda x: move_word(x, ' The', 0))

# dataset_test_sql.title = dataset_test_sql.title.apply(lambda x: move_word(x, ' A', 0))


# ------------------------------------------

# Replace (,The)
dataset_test_sql['title'].apply(lambda x: move_word(x, ' The', 0))

Toy Story
Jumanji
Grumpier Old Men
Waiting to Exhale
Father of the Bride Part II
Heat
Sabrina
Tom and Huck
Sudden Death
GoldenEye
 TheAmerican President
Dracula: Dead and Loving It
Balto
Nixon
Cutthroat Island
Casino
Sense and Sensibility
Four Rooms
Ace Ventura: When Nature Calls
Money Train
Get Shorty
Copycat
Assassins
Powder
Leaving Las Vegas
Othello
Now and Then
Persuasion
City of Lost Children The (Cité des enfants perdus La)
Shanghai Triad (Yao a yao yao dao waipo qiao)
Dangerous Minds
Twelve Monkeys (a.k.a. 12 Monkeys)
Wings of Courage
Babe
Carrington
Dead Man Walking
Across the Sea of Time
It Takes Two
Clueless
Cry the Beloved Country
Richard III
Dead Presidents
Restoration
Mortal Kombat
To Die For
How to Make an American Quilt
Seven (a.k.a. Se7en)
Pocahontas
When Night Is Falling
 TheUsual Suspects
Guardian Angel
Mighty Aphrodite
Lamerica
 TheBig Green
Georgia
Kids of the Round Table
Home for the Holidays
Postman The (Postino Il)
Confessional The (Confessionnal Le)
 TheIndian i

Flesh and Bone
Widows' Peak
For Love or Money
 TheFirm
Free Willy
Fresh
 TheFugitive
Geronimo: An American Legend
 TheGetaway
Getting Even with Dad
Go Fish
Good Man in Africa A
Guilty as Sin
Hard Target
Heaven & Earth
Hot Shots! Part Deux
Live Nude Girls
 TheEnglishman Who Went Up a Hill But Came Down a Mountain
 TheHouse of the Spirits
House Party 3
 TheHudsucker Proxy
I'll Do Anything
In the Army Now
In the Line of Fire
In the Name of the Father
 TheInkwell
What's Love Got to Do with It?
Jimmy Hollywood
Judgment Night
Jurassic Park
Kalifornia
Killing Zoe
King of the Hill
Lassie
Last Action Hero
Life with Mikey
Lightning Jack
M. Butterfly
Made in America
Malice
 TheMan Without a Face
Manhattan Murder Mystery
Menace II Society
Executive Decision
In the Realm of the Senses (Ai no corrida)
What Happened Was...
Much Ado About Nothing
Mr. Jones
Mr. Wonderful
Mrs. Doubtfire
Naked
 TheNext Karate Kid
 TheNew Age
No Escape
North
Orlando
Perfect World A
Philadelphia
 ThePiano
Poetic Justice
 T

Meet Me in St. Louis
 TheWizard of Oz
Gone with the Wind
My Favorite Year
Sunset Blvd. (a.k.a. Sunset Boulevard)
Citizen Kane
2001: A Space Odyssey
Golden Earrings
All About Eve
 TheWomen
Rebecca
Foreign Correspondent
Notorious
Spellbound
Affair to Remember An
To Catch a Thief
Father of the Bride
 TheBand Wagon
Ninotchka
Love in the Afternoon
Gigi
 TheReluctant Debutante
 TheAdventures of Robin Hood
 TheMark of Zorro
Laura
 TheGhost and Mrs. Muir
Lost Horizon
Top Hat
To Be or Not to Be
My Man Godfrey
Giant
East of Eden
 TheThin Man
His Girl Friday
Around the World in 80 Days
It's a Wonderful Life
Mr. Smith Goes to Washington
Bringing Up Baby
Penny Serenade
 TheScarlet Letter
Lady of Burlesque
Of Human Bondage
Angel on My Shoulder
Little Lord Fauntleroy
They Made Me a Criminal
 TheInspector General
Angel and the Badman
 The39 Steps
A Walk in the Sun
 TheOutlaw
Night of the Living Dead
 TheAfrican Queen
Beat the Devil
Cat on a Hot Tin Roof
 TheLast Time I Saw Paris
Meet John Doe
Algiers


101 Dalmatians
Forbidden Christ The (Cristo proibito Il)
I Can't Sleep (J'ai pas sommeil)
Die Hard 2
Star Trek: The Motion Picture
Star Trek VI: The Undiscovered Country
Star Trek V: The Final Frontier
Star Trek II: The Wrath of Khan
Star Trek III: The Search for Spock
Star Trek IV: The Voyage Home
Batman Returns
Young Guns
Young Guns II
Grease
Grease 2
Marked for Death
Adrenalin: Fear the Rush
 TheSubstance of Fire
Under Siege
Terror in a Texas Town
Jaws
Jaws 2
Jaws 3-D
My Fellow Americans
Mars Attacks!
Citizen Ruth
Jerry Maguire
Raising Arizona
Tin Men
Bastard Out of Carolina
In Love and War
Marvin's Room
Somebody is Waiting
Ghosts of Mississippi
Night Falls on Manhattan
Beavis and Butt-Head Do America
La Cérémonie
Scream
 TheLast of the Mohicans
Michael
Hamlet
Some Mother's Son
 TheWhole Wide World
Mother
Thieves (Voleurs Les)
Evita
 ThePortrait of a Lady
Walkabout
Hearts and Minds
Inside
Fierce Creatures
Zeus and Roxanne
Turbulence
Angel Baby
First Strike (Police Story 4: First Str

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [33]:
dataset_test_sql.title[10]

'American President  The'

# - - - - - - - - -  - - - - - - - - - - - - - - - - - -- - - - - - - - - - - - - - - - - -

##### Step 3: Clean the plot summary - 26.01.2020

In [130]:
dataset['plot'] = dataset['plot'].apply(lambda x: x.strip())

dataset['plot'] = dataset['plot'].apply(lambda x: x.replace(',', ''))

dataset['plot'] = dataset['plot'].apply(lambda x: x.replace('.', ''))

dataset['plot'] = dataset['plot'].apply(lambda x: x.replace('?', ''))

dataset['plot'] = dataset['plot'].apply(lambda x: x.replace('!', ''))

dataset['plot'] = dataset['plot'].apply(lambda x: x.replace('\n                    See full summary\xa0»', ''))

dataset['plot'] = dataset['plot'].apply(lambda x: x.rstrip())

In [131]:
dataset['plot'].iloc[-1]

'Story of a young woman journalist who becomes enmeshed in the politics of the annexation of Morrocco from Algeria at the turn of the twentieth century The focus is on her hardships because'

# - - - - - - - - -  - - - - - - - - - - - - - - - - - -- - - - - - - - - - - - - - - - - -

##### Step 4: Remove empty summaries - 26.01.2020

In [132]:
len(dataset[dataset['plot'] == 'Add a Plot »'])

176

In [97]:
dataset.shape

(49393, 10)

In [133]:
dataset = dataset[dataset['plot'] != 'Add a Plot »']

dataset.shape

(49217, 10)

In [ ]:
# Fix it with Jack

# def clean_plot_summary(plot):
#     for ch in ['\\','...','`','*','_','{','}','[',']','(',')','>','#','+','-','.','!','$','\'','?','/',"'s","\n                    See full summary\xa0»"]:
#         if ch in plot:
#             plot = plot.replace(ch, '\\'+ch)
#             return plot
#         else:
#             return plot

# - - - - - - - - -  - - - - - - - - - - - - - - - - - -- - - - - - - - - - - - - - - - - -

##### Step 5: Clean the similar movies (based on the similar actors they have) - 29.10.2019

In [ ]:
# dataset['Actors'] = dataset['Actors'].apply(lambda x: ', '.join(x)) old

In [ ]:
# dataset['Actors'] = dataset['Actors'].apply(lambda x: x.replace('-', ' ')) old replacement

In [99]:
dataset[dataset['title'].str.contains('Avatar')]

# I don't have duplicate movies since the dataset is a generic one from MovieLens

,title,genres,rating,imdb_url,reviews_url,actors,plot,imdb_rating,director,reviews
14449,Avatar,"[Action, Adventure, Sci-Fi]",3.65,http://www.imdb.com/title/tt0499549/,http://www.imdb.com/title/tt0499549/reviews?sp...,"[Sam Worthington, Zoe Saldana, Sigourney Weave...",A paraplegic Marine dispatched to the moon Pan...,7.8,James Cameron,"[Well, I just saw Avatar this morning, one of ..."


In [ ]:
# dataset_duplicated = dataset

# dataset_duplicated[dataset_duplicated.duplicated(['Actors'], keep='last')] old 

In [ ]:
# dataset = dataset.drop_duplicates('Actors', keep='first') old

In [ ]:
# dataset[dataset['title'].str.contains("Mask")]

**Comment:** The dataset does not have any duplicate movie title.

# - - - - - - - - -  - - - - - - - - - - - - - - - - - -- - - - - - - - - - - - - - - - - -

##### Step 6: Create the positive/negative column - 26.01.2020

In [134]:
dataset['sentiment_value'] = dataset['rating']

dataset['sentiment_value'].loc[dataset.rating >=3] = 1

dataset['sentiment_value'].loc[dataset.rating <3] = 0

c:\users\spano\appdata\local\programs\python\python37\lib\site-packages\pandas\core\indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [135]:
dataset.tail()

,title,genres,rating,imdb_url,reviews_url,actors,plot,imdb_rating,director,reviews,sentiment_value
49387,Dad on the run,[Comedy],3.0,http://www.imdb.com/title/tt0224807/,http://www.imdb.com/title/tt0224807/reviews?sp...,"[Clément Sibony, Rona Hartner, Isaac Sharry, M...",Fueled by klezmer and set in the Paris night w...,6.0,Dante Desarthe,"[A young father, charged with burying the fore...",1.0
49388,Üç Harfliler: Beddua,[Horror],1.0,http://www.imdb.com/title/tt7738930/,http://www.imdb.com/title/tt7738930/reviews?sp...,"[Beyzanur Mete, Esma Soysal, Serife Ünsal, Giz...",A little girl called a demon spirit to take he...,7.2,Alper Mestçi,[I like Alper Mestci movies. This film is like...,0.0
49390,Tempting Fate,"[Drama, Sci-Fi, Sport]",3.0,http://www.imdb.com/title/tt0137156/,http://www.imdb.com/title/tt0137156/reviews?sp...,"[Tate Donovan, Abraham Benrubi, Matt Craven, P...",A man who is still mourning the death of the w...,6.0,Peter Werner,[This film was not that bad but it would have ...,1.0
49391,"Spring Night, Summer Night",[Drama],3.5,http://www.imdb.com/title/tt0247527/,http://www.imdb.com/title/tt0247527/reviews?sp...,"[Larue Hall, Ted Heimerdinger, Marjorie Johnso...",Jessica the eldest daughter of a coal miner-tu...,7.0,Joseph L. Anderson,[Here's a film richly deserving of wider expos...,1.0
49392,Isabelle Eberhardt,"[Biography, Drama, History]",3.0,http://www.imdb.com/title/tt0102130/,http://www.imdb.com/title/tt0102130/reviews?sp...,"[Mathilda May, Tchéky Karyo, Peter O'Toole, Ri...",Story of a young woman journalist who becomes ...,5.9,Ian Pringle,[The story of the brief life (1877-1904) of Fr...,1.0


# - - - - - - - - -  - - - - - - - - - - - - - - - - - -- - - - - - - - - - - - - - - - - -

##### Step 7 Create the column Combined Features (29.10.2019)

In [136]:
# The columns that will be used to create the column "Combined Features"

def combine_features(row):
    return row['title'].lower() + " " + ' '.join(map(str, row['actors'])).lower() + " " + row['director'].lower() + " " + row['plot'].lower() + " " + ' '.join(row['genres']).lower()

# def combine_actors(row):
#     return ', '.join(row['Actors'])

dataset["movie_features"] = dataset.apply(combine_features, axis=1)

# dataset['Actors_Embeddings'] = dataset.apply(combine_actors, axis=1)

# Note: Instead of creating the Actor Embeddings column, vectorize the different name of each actor

In [137]:
dataset["movie_features"].iloc[0]

"toy story tom hanks tim allen don rickles jim varney wallace shawn john ratzenberger annie potts john morris erik von detten laurie metcalf r. lee ermey sarah freeman penn jillette jack angel spencer aste john lasseter a cowboy doll is profoundly threatened and jealous when a new spaceman figure supplants him as top toy in a boy's room adventure animation children"

# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -

#### Pickle the dataset (30.10.2019)

In [ ]:
# dataset.to_pickle('dataset_part_2_07112019.pkl') #previous version "dataset_30102019.pkl"

In [138]:
dataset.to_pickle('D:\\GitHub-Thesis\\58,000 movies\\dataset_part_2_25012020.pkl') #previous versions "dataset_part_2_07112019", "dataset_part_2_24122019"

In [139]:
dataset.shape

(49217, 12)

# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -

#### TO SQL SERVER

In [140]:
import pyodbc

dataset = pd.read_pickle('D:\\GitHub-Thesis\\58,000 movies\\dataset_part_2_25012020.pkl')

# dataset = dataset.reset_index(drop=True)

dataset.head()

,title,genres,rating,imdb_url,reviews_url,actors,plot,imdb_rating,director,reviews,sentiment_value,movie_features
0,Toy Story,"[Adventure, Animation, Children]",3.89,http://www.imdb.com/title/tt0114709/,http://www.imdb.com/title/tt0114709/reviews?sp...,"[Tom Hanks, Tim Allen, Don Rickles, Jim Varney...",A cowboy doll is profoundly threatened and jea...,8.3,John Lasseter,[Andy's toys live a reasonable life of fun and...,1.0,toy story tom hanks tim allen don rickles jim ...
1,Jumanji,"[Adventure, Children, Fantasy]",3.25,http://www.imdb.com/title/tt0113497/,http://www.imdb.com/title/tt0113497/reviews?sp...,"[Robin Williams, Jonathan Hyde, Kirsten Dunst,...",When two kids find and play a magical board ga...,7.0,Joe Johnston,[The smash hit 1995 film Jumanji- based on the...,1.0,jumanji robin williams jonathan hyde kirsten d...
2,Grumpier Old Men,"[Comedy, Romance]",3.17,http://www.imdb.com/title/tt0113228/,http://www.imdb.com/title/tt0113228/reviews?sp...,"[Walter Matthau, Jack Lemmon, Sophia Loren, An...",John and Max resolve to save their beloved bai...,6.7,Howard Deutch,[Sequels are rarely half as good as the origin...,1.0,grumpier old men walter matthau jack lemmon so...
3,Waiting to Exhale,"[Comedy, Drama, Romance]",2.87,http://www.imdb.com/title/tt0114885/,http://www.imdb.com/title/tt0114885/reviews?sp...,"[Whitney Houston, Angela Bassett, Loretta Devi...",Based on Terry McMillan's novel this film foll...,5.9,Forest Whitaker,[I read some of the other comments comcerning ...,0.0,waiting to exhale whitney houston angela basse...
4,Father of the Bride Part II,[Comedy],3.08,http://www.imdb.com/title/tt0113041/,http://www.imdb.com/title/tt0113041/reviews?sp...,"[Steve Martin, Diane Keaton, Martin Short, Kim...",George Banks must deal not only with the pregn...,6.0,Charles Shyer,"[I enjoyed this film, as I did Father of the B...",1.0,father of the bride part ii steve martin diane...


In [ ]:
#dataset_test_sql = dataset.iloc[0:20]

In [141]:
dataset[dataset.isna().any(axis=1)]

,title,genres,rating,imdb_url,reviews_url,actors,plot,imdb_rating,director,reviews,sentiment_value,movie_features


In [142]:
dataset.isnull().sum()

title              0
genres             0
rating             0
imdb_url           0
reviews_url        0
actors             0
plot               0
imdb_rating        0
director           0
reviews            0
sentiment_value    0
movie_features     0
dtype: int64

In [151]:
# MOVIE CONTENT TO SQL TABLE

connStr = pyodbc.connect("DRIVER={SQL Server Native Client 11.0};" #SQL Server Native Client 11.0, #{ODBC Driver 17 for SQL Server}
                         "SERVER=LAPTOP-IFTEP7AL;"
                         "DATABASE=Movies_Dataset;"
                         "Trusted_Connection=Yes")

cursor = connStr.cursor()
 
delete_table =  """           
                        IF [Movies_Dataset].[dbo].TableExists('table_movies') = 1
                             DELETE FROM table_movies
                """

cursor.execute(delete_table)

insert_values = """
                        EXEC [Movies_Dataset].[dbo].[store_movies] @Movie_Title = ?, @IMDB_Rating = ?, @Director = ?, @Plot_Summary = ?, 
                        
                        @IMDB_Url = ?, @Reviews_Url = ?, @MovieLens_User_Rating = ?, @Sentiment_value = ?;
                """

for index, row in dataset.iterrows():

    params = (row['title'], row['imdb_rating'], row['director'], row['plot'], row['imdb_url'], row['reviews_url'], row['rating'], row['sentiment_value'])
    
    cursor.execute(insert_values, params)

connStr.commit()

cursor.close()

connStr.close()

ProgrammingError: ('42000', '[42000] [Microsoft][SQL Server Native Client 11.0][SQL Server]Error converting data type float to numeric. (8114) (SQLExecDirectW)')

In [144]:
# ACTORS TO SQL TABLE

column_actors = dataset.loc[:, ['title', 'actors']]

column_actors.shape

s = column_actors.apply(lambda x: pd.Series(x['actors']), axis=1).stack().reset_index(level=1, drop=True)

s.name = 'actors'

actor_names = column_actors.drop('actors', axis=1).join(s)

actor_names['actors'] = pd.Series(actor_names['actors'], dtype=object)

print(actor_names.head(15))

print(actor_names.shape)

       title             actors
0  Toy Story          Tom Hanks
0  Toy Story          Tim Allen
0  Toy Story        Don Rickles
0  Toy Story         Jim Varney
0  Toy Story      Wallace Shawn
0  Toy Story  John Ratzenberger
0  Toy Story        Annie Potts
0  Toy Story        John Morris
0  Toy Story    Erik von Detten
0  Toy Story     Laurie Metcalf
0  Toy Story       R. Lee Ermey
0  Toy Story      Sarah Freeman
0  Toy Story      Penn Jillette
0  Toy Story         Jack Angel
0  Toy Story       Spencer Aste
(656454, 2)


In [145]:
connStr = pyodbc.connect("DRIVER={SQL Server Native Client 11.0};"
                         "SERVER=LAPTOP-IFTEP7AL;"
                         "DATABASE=Movies_Dataset;"
                         "Trusted_Connection=yes")

cursor = connStr.cursor()
 
delete_table =  """           
                        IF dbo.TableExists('table_actors') = 1
                             DELETE FROM table_actors      
                """

insert_values = """ 
                        EXEC [dbo].[store_actors] @Movie_Title = ?, @Actors = ?;
                """   

cursor.execute(delete_table)

for index, row in actor_names.iterrows():

    params = (row['title'], row['actors'])
    cursor.execute(insert_values, params)

connStr.commit()

cursor.close()

connStr.close()

In [146]:
# GENRES TO SQL TABLE

column_genres = dataset.loc[:, ['title', 'genres']]

s = column_genres.apply(lambda x: pd.Series(x['genres']), axis=1).stack().reset_index(level=1, drop=True)

s.name = 'genres'

genres = column_genres.drop('genres', axis=1).join(s)

genres['genres'] = pd.Series(genres['genres'], dtype=object)

genres.head(15)

,title,genres
0,Toy Story,Adventure
0,Toy Story,Animation
0,Toy Story,Children
1,Jumanji,Adventure
1,Jumanji,Children
1,Jumanji,Fantasy
2,Grumpier Old Men,Comedy
2,Grumpier Old Men,Romance
3,Waiting to Exhale,Comedy
3,Waiting to Exhale,Drama


In [147]:
connStr = pyodbc.connect("DRIVER={SQL Server Native Client 11.0};"
                         "SERVER=LAPTOP-IFTEP7AL;"
                         "DATABASE=Movies_Dataset;"
                         "Trusted_Connection=yes")

cursor = connStr.cursor()
 
delete_table =  """           
                        IF dbo.TableExists('table_genres') = 1
                             DELETE FROM table_genres     
                """

insert_values = """ 
                        EXEC [dbo].[store_genres] @Movie_Title = ?, @Genres = ?;
                """   

cursor.execute(delete_table)

for index, row in genres.iterrows():

    params = (row['title'], row['genres'])
    
    cursor.execute(insert_values, params)

connStr.commit()

cursor.close()

connStr.close()

In [148]:
# REVIEWS TO SQL TABLE

column_reviews = dataset.loc[:, ['title', 'reviews']]

s = column_reviews.apply(lambda x: pd.Series(x['reviews']), axis=1).stack().reset_index(level=1, drop=True)

s.name = 'reviews'

reviews = column_reviews.drop('reviews', axis=1).join(s)

reviews['reviews'] = pd.Series(reviews['reviews'], dtype=object)

reviews.head(15)

,title,reviews
0,Toy Story,Andy's toys live a reasonable life of fun and ...
0,Toy Story,I am a big fan of the animated movies coming f...
0,Toy Story,This is a very clever animated story that was ...
0,Toy Story,"Y'know, I always suspected that my toys were c..."
0,Toy Story,Toy Story is the film that started Pixar Anima...
0,Toy Story,Toy Story is a sheer delight to view on the sc...
0,Toy Story,Toy Story is not only the best Disney film bec...
0,Toy Story,Though I am not a big fan of computer animatio...
0,Toy Story,Just in case you were also wondering what happ...
0,Toy Story,"Toy story was a fun, imaginative renewal of th..."


In [149]:
connStr = pyodbc.connect("DRIVER={SQL Server Native Client 11.0};"
                         "SERVER=LAPTOP-IFTEP7AL;"
                         "DATABASE=Movies_Dataset;"
                         "Trusted_Connection=yes")

cursor = connStr.cursor()
 
delete_table =  """           
                        IF dbo.TableExists('table_reviews') = 1
                             DELETE FROM table_reviews      
                """

insert_values = """ 
                        EXEC [dbo].[store_reviews] @Movie_Title = ?, @Reviews = ?;
                """   

cursor.execute(delete_table)

for index, row in reviews.iterrows():

    params = (row['title'], row['reviews'])
    cursor.execute(insert_values, params)

connStr.commit()

cursor.close()

connStr.close()

#### End of part 2 (Transforming the columns of the final dataset)